In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pywt
import itertools
import warnings
warnings.filterwarnings("ignore")

import gc
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, LSTM, GRU, SimpleRNN, Conv1D,
                                     MaxPooling1D, Flatten, Input, Reshape,
                                     Lambda, concatenate, TimeDistributed)
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import (mean_squared_error, mean_absolute_error, 
                             r2_score, explained_variance_score)
from sklearn.model_selection import TimeSeriesSplit, ParameterGrid
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import pickle
import json
from datetime import datetime
import glob

# ================================================
# 🚀 ANÁLISIS OPTIMIZADO - VERSIÓN 8-12 HORAS
# ================================================

print("🚀 CONFIGURACIÓN OPTIMIZADA PARA 8-12 HORAS")
print("=" * 50)

# Setup directories
os.makedirs('plots', exist_ok=True)
os.makedirs('plots/regimes', exist_ok=True)
os.makedirs('results', exist_ok=True)

# ⚡ CONFIGURACIÓN OPTIMIZADA - REDUCIDA
param_grid_optimized = {
    'ANN': {
        'layers': [[64,32], [32,16]],          # Solo 2 configuraciones
        'learning_rate': [1e-3],               # Solo 1 LR
        'batch_size': [32]                     # Solo 1 batch size
    }
}

# ================================================
# 1. DEFINICIÓN DE REGÍMENES DE MERCADO
# ================================================
market_periods = {
    'bull': ('2012-10-05','2018-01-01'),
    'bear': ('2018-01-02','2020-03-16'),
    'recovery': ('2020-03-17','2025-03-27')
}

def label_market_regime(date):
    """Etiquetar regímenes de mercado basado en fecha"""
    if isinstance(date, (int, float)):
        date = pd.to_datetime(date, unit='ns' if date > 1e15 else 's')
    elif not hasattr(date, 'strftime'):
        date = pd.to_datetime(date)
    
    ds = date.strftime('%Y-%m-%d')
    for regime, (start, end) in market_periods.items():
        if start <= ds <= end:
            return regime
    return 'other'

def get_regime_data(df, regime):
    """Obtener datos específicos de un régimen"""
    df_regime = df.copy()
    df_regime['regime'] = df_regime.index.to_series().apply(label_market_regime)
    return df_regime[df_regime['regime'] == regime].copy()

# ================================================
# 2. CARGA Y PREPARACIÓN DE DATOS
# ================================================
def load_data(path):
    """Cargar datos de mercado"""
    df = pd.read_csv(path, parse_dates=['DATE'])
    df.columns = df.columns.str.upper()
    df.set_index('DATE', inplace=True)
    return df

# ================================================
# 3. MÉTODOS FRACTALES OPTIMIZADOS
# ================================================
def hurst_exponent_fast(ts):
    """Calcular exponente de Hurst (versión rápida)"""
    if len(ts) < 10:
        return np.nan
    
    lags = range(2, min(10, len(ts)//3))  # Reducido de 20 a 10
    tau = [np.std(ts[lag:] - ts[:-lag]) for lag in lags if lag < len(ts)]
    
    if len(tau) < 2:
        return np.nan
    try:
        poly = np.polyfit(np.log(range(2, 2+len(tau))), np.log(tau), 1)
        return poly[0]
    except:
        return np.nan

def apply_hurst_fast(df, price_col='VIX', window_size=30):
    """Aplicar análisis de Hurst (versión rápida)"""
    df_result = df.copy()
    df_result['HURST_' + price_col] = df_result[price_col].rolling(
        window=window_size, min_periods=5  # Reducido min_periods
    ).apply(hurst_exponent_fast, raw=True)
    return df_result

def apply_wavelet_energy_fast(segment, wavelet='db2', level=2):
    """Aplicar energía wavelet (versión rápida)"""
    if len(segment) < 2**(level+1):
        return [np.nan] * (level + 1)
    try:
        coeffs = pywt.wavedec(segment, wavelet, level=level)
        return [np.sum(c**2) for c in coeffs]
    except:
        return [np.nan] * (level + 1)

def apply_wavelets_fast(df, col_list=None, window=50):
    """Aplicar transformadas wavelet (versión optimizada)"""
    if col_list is None:
        col_list = ['VIX']  # Solo VIX para rapidez
    
    df_result = df.copy()
    wavelet_cols = []
    
    # 🚀 OPTIMIZACIÓN: Calcular solo cada 5 filas para reducir tiempo
    step_size = 5
    
    for col in col_list:
        if col not in df_result.columns:
            continue
            
        feats = []
        for i in range(0, len(df_result), step_size):
            start_idx = max(0, i - window + 1)
            segment = df_result[col].iloc[start_idx:i+1].dropna()
            
            if len(segment) >= 6:  # Reducido mínimo
                energy = apply_wavelet_energy_fast(segment.values)
            else:
                energy = [np.nan] * 3
            feats.append(energy)
        
        # Interpolar valores faltantes
        for j in range(len(df_result)):
            idx = j // step_size
            if idx >= len(feats):
                idx = len(feats) - 1
        
        # Crear columnas wavelet
        full_feats = []
        for i in range(len(df_result)):
            idx = min(i // step_size, len(feats) - 1)
            full_feats.append(feats[idx])
        
        for j in range(3):
            new_col = f'WAVELET_{col}_L{j}'
            df_result[new_col] = [x[j] for x in full_feats]
            wavelet_cols.append(new_col)
    
    return df_result, wavelet_cols

# ================================================
# 4. PREPARACIÓN DE FEATURES
# ================================================
def prepare_features_regime_fast(df_regime, features, target='VIX', lookback=5, scale_method='MinMax'):
    """Preparar features (versión rápida con lookback reducido)"""
    print(f"   Preparando features para régimen con {len(df_regime)} observaciones")
    
    # Limpiar datos
    df_clean = df_regime.dropna(subset=features + [target]).copy()
    
    if len(df_clean) <= lookback:
        print(f"   ❌ Datos insuficientes: {len(df_clean)} <= {lookback}")
        return None, None, None, None
    
    # Escalado
    scaler = StandardScaler() if scale_method == 'Standard' else MinMaxScaler()
    scaled = scaler.fit_transform(df_clean[features + [target]])
    
    # Crear secuencias (cada 2 puntos para reducir tamaño)
    X, y, idx = [], [], []
    for i in range(lookback, len(scaled), 2):  # Step=2 para reducir datos
        X.append(scaled[i-lookback:i, :-1])
        y.append(scaled[i, -1])
        idx.append(df_clean.index[i])
    
    print(f"   ✅ Secuencias creadas: {len(X)}")
    return np.array(X), np.array(y), idx, scaler

# ================================================
# 5. CONSTRUCCIÓN DE MODELOS SIMPLIFICADOS
# ================================================
def build_model_fast(model_type, input_shape, layers=None, lr=1e-3):
    """Construir modelo (versión rápida y simple)"""
    model = Sequential()
    
    if layers is None:
        configs = {
            'ANN': [32, 16], 'LSTM': [32], 'GRU': [32], 'CNN': [32]
        }
        layers = configs.get(model_type, [32, 16])
    
    if model_type == 'ANN':
        model.add(Input(shape=input_shape))
        model.add(Flatten())
        for units in layers:
            model.add(Dense(units, activation='relu'))
        model.add(Dense(1))
        
    elif model_type == 'LSTM':
        model.add(LSTM(layers[0], input_shape=input_shape))
        model.add(Dense(1))
        
    elif model_type == 'GRU':
        model.add(GRU(layers[0], input_shape=input_shape))
        model.add(Dense(1))
        
    elif model_type == 'CNN':
        model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
        model.add(MaxPooling1D(2))
        model.add(Flatten())
        model.add(Dense(16, activation='relu'))
        model.add(Dense(1))
    
    optimizer = Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# ================================================
# 6. MÉTRICAS Y EVALUACIÓN
# ================================================
def calculate_metrics(y_true, y_pred):
    """Calcular métricas de evaluación"""
    return {
        'mse': mean_squared_error(y_true, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred),
        'explained_variance': explained_variance_score(y_true, y_pred),
        'mape': np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100
    }

# ================================================
# 7. ENTRENAMIENTO OPTIMIZADO
# ================================================
def train_statistical_model_regime_fast(df_regime, model_type, target='VIX'):
    """Entrenar modelos estadísticos (versión rápida)"""
    series = df_regime[target].dropna()
    
    if len(series) < 15:  # Reducido de 20 a 15
        return None, None, None
    
    split = int(len(series) * 0.8)
    train_series = series.iloc[:split]
    test_series = series.iloc[split:]
    
    try:
        if model_type == 'ARIMA':
            # ARIMA más simple para rapidez
            model = ARIMA(train_series, order=(1, 0, 1))
            fitted_model = model.fit()
            pred = fitted_model.forecast(steps=len(test_series))
            pred_values = pred.values if hasattr(pred, 'values') else pred
            
        elif model_type == 'GARCH':
            model = arch_model(train_series, vol='Garch', p=1, q=1, dist='normal')
            fitted_model = model.fit(disp='off')
            forecast = fitted_model.forecast(horizon=len(test_series), reindex=False)
            pred_values = np.sqrt(forecast.variance.values[-1])
            if isinstance(pred_values, (int, float)):
                pred_values = np.full(len(test_series), pred_values)
        
        metrics = calculate_metrics(test_series.values, pred_values)
        pred_df = pd.DataFrame({
            'time_index': test_series.index,
            'y_true': test_series.values,
            'y_pred': pred_values
        })
        
        return metrics, pred_df, fitted_model
        
    except Exception as e:
        print(f"      Error en {model_type}: {e}")
        return None, None, None

def train_ml_model_regime_fast(X, y, idx, model_type, regime_name, epochs=15, batch_size=32):
    """Entrenar modelos ML (versión rápida)"""
    if len(X) < 15:  # Reducido de 20 a 15
        print(f"      ❌ Datos insuficientes para ML: {len(X)}")
        return None, None
    
    # División temporal
    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]
    idx_test = idx[split:]
    
    try:
        # Grid search SIMPLIFICADO para ANN
        if model_type == 'ANN' and model_type in param_grid_optimized:
            print(f"      🔍 Grid search rápido para {model_type}")
            best_cfg, best_loss = None, np.inf
            
            for cfg in ParameterGrid(param_grid_optimized[model_type]):
                try:
                    temp_model = build_model_fast(model_type, X_train.shape[1:], 
                                                 layers=cfg['layers'], lr=cfg['learning_rate'])
                    hist = temp_model.fit(X_train, y_train, epochs=10,  # Reducido epochs
                                        batch_size=cfg['batch_size'],
                                        validation_split=0.2, verbose=0,
                                        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])
                    val_loss = min(hist.history['val_loss']) if 'val_loss' in hist.history else hist.history['loss'][-1]
                    
                    if val_loss < best_loss:
                        best_loss, best_cfg = val_loss, cfg
                    
                    K.clear_session()
                    del temp_model
                    gc.collect()
                    
                except Exception as e:
                    print(f"        Error en configuración {cfg}: {e}")
                    continue
            
            if best_cfg:
                model = build_model_fast(model_type, X_train.shape[1:], 
                                        layers=best_cfg['layers'], lr=best_cfg['learning_rate'])
                batch_size = best_cfg['batch_size']
            else:
                model = build_model_fast(model_type, X_train.shape[1:])
                
        else:
            model = build_model_fast(model_type, X_train.shape[1:])
        
        # Entrenamiento final
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                          validation_split=0.2, verbose=0,
                          callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])  # Reducido patience
        
        # Predicción
        y_pred = model.predict(X_test, verbose=0).flatten()
        
        # Métricas
        metrics = calculate_metrics(y_test, y_pred)
        metrics['train_loss'] = history.history['loss'][-1]
        metrics['val_loss'] = history.history['val_loss'][-1] if 'val_loss' in history.history else np.nan
        
        # DataFrame de predicciones
        pred_df = pd.DataFrame({
            'time_index': idx_test,
            'y_true': y_test,
            'y_pred': y_pred
        })
        
        # Limpiar memoria
        K.clear_session()
        del model
        gc.collect()
        
        return metrics, pred_df
        
    except Exception as e:
        print(f"      ❌ Error en {model_type}: {e}")
        return None, None

# ================================================
# 8. SISTEMA DE CHECKPOINTS SIMPLIFICADO
# ================================================
def save_checkpoint_fast(results, predictions, regime_summaries, current_regime, current_combo_idx, total_combos):
    """Guardar checkpoint (versión ligera)"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    checkpoint_data = {
        'timestamp': timestamp,
        'current_regime': current_regime,
        'current_combo_idx': current_combo_idx,
        'total_combos': total_combos,
        'results': results,
        'predictions': predictions,
        'regime_summaries': regime_summaries,
        'progress_pct': (current_combo_idx / total_combos) * 100 if total_combos > 0 else 0
    }
    
    checkpoint_file = f'results/checkpoint_fast_{current_regime}_{timestamp}.pkl'
    with open(checkpoint_file, 'wb') as f:
        pickle.dump(checkpoint_data, f)
    
    print(f"   💾 Checkpoint guardado: {checkpoint_file}")
    return checkpoint_file

def load_latest_checkpoint_fast():
    """Cargar el checkpoint más reciente"""
    checkpoint_files = glob.glob('results/checkpoint_fast_*.pkl')
    if not checkpoint_files:
        print("   ℹ️ No se encontraron checkpoints previos")
        return None
    
    latest_file = max(checkpoint_files, key=os.path.getctime)
    
    try:
        with open(latest_file, 'rb') as f:
            checkpoint_data = pickle.load(f)
        
        print(f"   ✅ Checkpoint cargado: {latest_file}")
        print(f"   📊 Progreso anterior: {checkpoint_data['progress_pct']:.1f}%")
        print(f"   📈 Resultados previos: {len(checkpoint_data['results'])}")
        
        return checkpoint_data
    
    except Exception as e:
        print(f"   ❌ Error cargando checkpoint: {e}")
        return None

def should_resume_from_checkpoint_fast():
    """Preguntar al usuario si quiere resumir desde checkpoint"""
    checkpoint = load_latest_checkpoint_fast()
    if checkpoint is None:
        return False, None
    
    print(f"\n🔄 CHECKPOINT ENCONTRADO:")
    print(f"   📅 Fecha: {checkpoint['timestamp']}")
    print(f"   🎪 Régimen: {checkpoint['current_regime']}")
    print(f"   📊 Progreso: {checkpoint['progress_pct']:.1f}%")
    print(f"   📈 Resultados: {len(checkpoint['results'])}")
    
    while True:
        response = input("\n¿Desea continuar desde el checkpoint? (y/n): ").lower().strip()
        if response in ['y', 'yes', 's', 'si']:
            return True, checkpoint
        elif response in ['n', 'no']:
            return False, None
        else:
            print("Por favor responda 'y' o 'n'")

# ================================================
# 9. BENCHMARK OPTIMIZADO PARA 8-12 HORAS
# ================================================
def benchmark_optimized():
    """Benchmark optimizado para 8-12 horas de ejecución"""
    
    print("🚀 INICIANDO BENCHMARK OPTIMIZADO (8-12 HORAS)")
    print("=" * 50)
    
    # ✨ VERIFICAR CHECKPOINTS
    resume, checkpoint_data = should_resume_from_checkpoint_fast()
    
    # 🔥 CONFIGURACIÓN OPTIMIZADA
    base_cols = ['DIX', 'GEX', 'SKEW']           # 3 features principales (reducido de 4)
    statistical_models = ['ARIMA']               # Solo ARIMA (GARCH muy lento)
    ml_models = ['ANN', 'LSTM', 'GRU']          # 3 modelos principales (sin CNN, CapsNet)
    regimes = ['bull', 'bear', 'recovery']      # Los 3 regímenes
    fractal_methods = ['none', 'hurst']         # Sin wavelets inicialmente (muy lento)
    
    # Combinaciones de features (reducido)
    feature_combinations = []
    for r in range(1, len(base_cols) + 1):
        if r <= 2:  # Solo combinaciones de 1 y 2 features
            feature_combinations.extend(itertools.combinations(base_cols, r))
    
    print(f"📊 CONFIGURACIÓN OPTIMIZADA:")
    print(f"   Features: {base_cols}")
    print(f"   Modelos ML: {ml_models}")
    print(f"   Modelos Estadísticos: {statistical_models}")
    print(f"   Métodos Fractales: {fractal_methods}")
    print(f"   Combinaciones de Features: {len(feature_combinations)}")
    
    total_experiments = len(regimes) * len(feature_combinations) * len(fractal_methods) * (len(ml_models) + len(statistical_models))
    print(f"   🎯 Total Experimentos: {total_experiments}")
    print(f"   ⏱️ Tiempo Estimado: 8-12 horas")
    
    # 🔄 INICIALIZAR CONTENEDORES
    if resume and checkpoint_data:
        all_results = checkpoint_data['results']
        all_predictions = checkpoint_data['predictions'] 
        regime_summaries = checkpoint_data['regime_summaries']
        start_regime_idx = regimes.index(checkpoint_data['current_regime']) if checkpoint_data['current_regime'] in regimes else 0
        start_combo_idx = checkpoint_data['current_combo_idx']
        print(f"   🔄 Resumiendo desde régimen {checkpoint_data['current_regime']}, combo {start_combo_idx}")
    else:
        all_results = []
        all_predictions = []
        regime_summaries = []
        start_regime_idx = 0
        start_combo_idx = 0
        print("   🆕 Iniciando análisis completo desde el principio")
    
    # Cargar datos
    print("\n📊 Cargando datos...")
    df0 = load_data(r'C:\Users\antonio-jose.martine\OneDrive - GFI\Documentos\Doctorado\articulo 8 peer review\Data\merged_market_data_vix.csv')
    print(f"   Datos cargados: {len(df0)} observaciones desde {df0.index.min()} hasta {df0.index.max()}")
    
    # Cálculos para progress tracking
    total_regime_combos = len(feature_combinations) * len(fractal_methods)
    total_global_combos = len(regimes) * total_regime_combos
    
    # Análisis por régimen
    for regime_idx, regime in enumerate(regimes):
        if regime_idx < start_regime_idx:
            continue
            
        print(f"\n🎪 ANALIZANDO RÉGIMEN: {regime.upper()}")
        print("-" * 40)
        
        # Obtener datos del régimen
        df_regime = get_regime_data(df0, regime)
        
        if len(df_regime) < 30:  # Reducido mínimo
            print(f"   ❌ Datos insuficientes para {regime}: {len(df_regime)} observaciones")
            continue
        
        period_start, period_end = market_periods[regime]
        print(f"   📅 Período: {period_start} a {period_end}")
        print(f"   📊 Observaciones: {len(df_regime)}")
        
        # Estadísticas del régimen
        if not resume or regime not in [rs['regime'] for rs in regime_summaries]:
            vix_stats = {
                'regime': regime,
                'start_date': period_start,
                'end_date': period_end,
                'n_observations': len(df_regime),
                'vix_mean': df_regime['VIX'].mean(),
                'vix_std': df_regime['VIX'].std(),
                'vix_min': df_regime['VIX'].min(),
                'vix_max': df_regime['VIX'].max(),
                'vix_median': df_regime['VIX'].median()
            }
            regime_summaries.append(vix_stats)
        
        # Determinar punto de inicio
        combo_start_idx = start_combo_idx if regime_idx == start_regime_idx else 0
        
        # Análisis por combinaciones
        for combo_idx, combo in enumerate(feature_combinations):
            if regime_idx == start_regime_idx and combo_idx < combo_start_idx // len(fractal_methods):
                continue
                
            for fractal_idx, fractal in enumerate(fractal_methods):
                if regime_idx == start_regime_idx and (combo_idx * len(fractal_methods) + fractal_idx) < start_combo_idx:
                    continue
                
                # Progreso
                global_combo_idx = (regime_idx * total_regime_combos + 
                                  combo_idx * len(fractal_methods) + fractal_idx)
                progress = (global_combo_idx / total_global_combos) * 100
                
                print(f"\n   🔬 Features: {'+'.join(combo)} | Fractal: {fractal} ({progress:.1f}%)")
                
                try:
                    # Preparar datos con fractales
                    df_work = df_regime.copy()
                    
                    if fractal == 'hurst':
                        df_work = apply_hurst_fast(df_work, 'VIX')
                    elif fractal == 'wavelet':
                        df_work, wavelet_cols = apply_wavelets_fast(df_work, ['VIX'])
                    
                    # Features finales
                    features = list(combo)
                    if fractal == 'hurst':
                        features.append('HURST_VIX')
                    elif fractal == 'wavelet':
                        features.extend([c for c in df_work.columns if c.startswith('WAVELET_')])
                    
                    # Verificar features
                    available_features = [f for f in features if f in df_work.columns]
                    if len(available_features) != len(features):
                        missing = set(features) - set(available_features)
                        print(f"      ⚠️ Features faltantes: {missing}")
                        continue
                    
                    features = available_features
                    
                    # Limpiar datos
                    df_clean = df_work.dropna(subset=features + ['VIX'])
                    if len(df_clean) < 20:  # Reducido mínimo
                        print(f"      ❌ Datos insuficientes: {len(df_clean)}")
                        continue
                    
                    # Modelos estadísticos
                    for model_type in statistical_models:
                        print(f"      📊 Entrenando {model_type}...")
                        
                        metrics, pred_df, fitted_model = train_statistical_model_regime_fast(
                            df_clean, model_type, 'VIX'
                        )
                        
                        if metrics is not None:
                            result_record = {
                                'regime': regime,
                                'features': '+'.join(combo),
                                'fractal': fractal,
                                'model': model_type,
                                'n_observations': len(df_clean),
                                **metrics
                            }
                            all_results.append(result_record)
                            
                            pred_df_regime = pred_df.copy()
                            pred_df_regime['regime'] = regime
                            pred_df_regime['features'] = '+'.join(combo)
                            pred_df_regime['fractal'] = fractal
                            pred_df_regime['model'] = model_type
                            
                            all_predictions.extend(pred_df_regime.to_dict('records'))
                            
                            print(f"        ✅ MSE: {metrics['mse']:.6f}, R²: {metrics['r2']:.4f}")
                        else:
                            print(f"        ❌ Falló")
                    
                    # Preparar datos para ML
                    X, y, idx, scaler = prepare_features_regime_fast(df_clean, features, 'VIX', lookback=5)
                    
                    if X is not None:
                        # Modelos ML
                        for model_type in ml_models:
                            print(f"      🤖 Entrenando {model_type}...")
                            
                            metrics, pred_df = train_ml_model_regime_fast(
                                X, y, idx, model_type, regime, epochs=15, batch_size=32
                            )
                            
                            if metrics is not None:
                                result_record = {
                                    'regime': regime,
                                    'features': '+'.join(combo),
                                    'fractal': fractal,
                                    'model': model_type,
                                    'n_observations': len(X),
                                    **metrics
                                }
                                all_results.append(result_record)
                                
                                pred_df_regime = pred_df.copy()
                                pred_df_regime['regime'] = regime
                                pred_df_regime['features'] = '+'.join(combo)
                                pred_df_regime['fractal'] = fractal
                                pred_df_regime['model'] = model_type
                                
                                all_predictions.extend(pred_df_regime.to_dict('records'))
                                
                                print(f"        ✅ MSE: {metrics['mse']:.6f}, R²: {metrics['r2']:.4f}")
                            else:
                                print(f"        ❌ Falló")
                    
                    # 💾 CHECKPOINT cada 5 combinaciones (más frecuente)
                    if (global_combo_idx + 1) % 5 == 0:
                        save_checkpoint_fast(all_results, all_predictions, regime_summaries, 
                                           regime, global_combo_idx + 1, total_global_combos)
                    
                except Exception as e:
                    print(f"      ❌ Error en combinación: {e}")
                    continue
                
                # Limpiar memoria
                gc.collect()
        
        # Reset para siguiente régimen
        start_combo_idx = 0
        
        # Checkpoint al finalizar régimen
        final_regime_combo = (regime_idx + 1) * total_regime_combos
        save_checkpoint_fast(all_results, all_predictions, regime_summaries, 
                           f"{regime}_COMPLETED", final_regime_combo, total_global_combos)
    
    # ================================================
    # GUARDAR RESULTADOS FINALES
    # ================================================
    print(f"\n💾 GUARDANDO RESULTADOS FINALES...")
    
    results_df = pd.DataFrame(all_results)
    results_df.to_csv('results/optimized_results.csv', index=False)
    print(f"   ✅ results/optimized_results.csv ({len(results_df)} filas)")
    
    predictions_df = pd.DataFrame(all_predictions)
    predictions_df.to_csv('results/optimized_predictions.csv', index=False)
    print(f"   ✅ results/optimized_predictions.csv ({len(predictions_df)} filas)")
    
    regime_summary_df = pd.DataFrame(regime_summaries)
    regime_summary_df.to_csv('results/optimized_regime_characteristics.csv', index=False)
    print(f"   ✅ results/optimized_regime_characteristics.csv")
    
    # Análisis rápido
    analyze_results_fast(results_df, regime_summary_df)
    
    print(f"\n✅ BENCHMARK OPTIMIZADO COMPLETADO")
    print(f"📊 Total experimentos: {len(results_df)}")
    print(f"🔮 Total predicciones: {len(predictions_df)}")
    
    return results_df, predictions_df, regime_summary_df

def analyze_results_fast(results_df, regime_summary_df):
    """Análisis rápido de resultados"""
    print(f"\n📊 ANÁLISIS RÁPIDO DE RESULTADOS:")
    print("-" * 40)
    
    # Mejor modelo por régimen
    print("\n🏆 MEJORES MODELOS POR RÉGIMEN:")
    for regime in results_df['regime'].unique():
        regime_results = results_df[results_df['regime'] == regime]
        best_result = regime_results.loc[regime_results['mse'].idxmin()]
        
        print(f"\n{regime.upper()}:")
        print(f"   Mejor: {best_result['model']} + {best_result['fractal']}")
        print(f"   Features: {best_result['features']}")
        print(f"   MSE: {best_result['mse']:.6f}")
        print(f"   R²: {best_result['r2']:.4f}")
    
    # Resumen por método fractal
    print(f"\n🔬 RESUMEN POR MÉTODO FRACTAL:")
    fractal_summary = results_df.groupby('fractal').agg({
        'mse': 'mean',
        'r2': 'mean'
    }).round(6)
    print(fractal_summary)
    
    # Top 5 mejores configuraciones
    print(f"\n🥇 TOP 5 MEJORES CONFIGURACIONES:")
    top5 = results_df.nsmallest(5, 'mse')[['regime', 'model', 'fractal', 'features', 'mse', 'r2']]
    for idx, row in top5.iterrows():
        print(f"   {row['regime']} | {row['model']} + {row['fractal']} | {row['features']} | MSE: {row['mse']:.6f} | R²: {row['r2']:.4f}")

# ================================================
# EJECUCIÓN PRINCIPAL OPTIMIZADA
# ================================================
if __name__ == '__main__':
    print("🚀 INICIANDO ANÁLISIS OPTIMIZADO - VERSIÓN 8-12 HORAS")
    print("=" * 60)
    
    # Mostrar configuración optimizada
    print("\n⚡ OPTIMIZACIONES APLICADAS:")
    print("  • Features reducidas: DIX, GEX, SKEW (3 en lugar de 4)")
    print("  • Solo ARIMA (sin GARCH)")
    print("  • Modelos ML: ANN, LSTM, GRU (sin CNN, CapsNet)")
    print("  • Fractales: none, hurst (sin wavelets inicialmente)")
    print("  • Epochs reducidos: 15 (en lugar de 30)")
    print("  • Lookback reducido: 5 (en lugar de 10)")
    print("  • Grid search simplificado: 2 configuraciones ANN")
    print("  • Checkpoints cada 5 combinaciones")
    print("  • Sampling de datos cada 2 puntos para ML")
    
    try:
        # Ejecutar benchmark optimizado
        results_df, predictions_df, regime_summary_df = benchmark_optimized()
        
        # Limpiar memoria final
        gc.collect()
        
        print(f"\n🎉 ANÁLISIS OPTIMIZADO COMPLETADO EXITOSAMENTE")
        print(f"📊 Resultados guardados en: results/optimized_*.csv")
        
    except KeyboardInterrupt:
        print(f"\n⚠️ ANÁLISIS INTERRUMPIDO")
        print(f"💾 Los checkpoints permiten continuar desde donde se quedó")
        
    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        print(f"💾 Revisa los checkpoints en results/")

🚀 CONFIGURACIÓN OPTIMIZADA PARA 8-12 HORAS
🚀 INICIANDO ANÁLISIS OPTIMIZADO - VERSIÓN 8-12 HORAS

⚡ OPTIMIZACIONES APLICADAS:
  • Features reducidas: DIX, GEX, SKEW (3 en lugar de 4)
  • Solo ARIMA (sin GARCH)
  • Modelos ML: ANN, LSTM, GRU (sin CNN, CapsNet)
  • Fractales: none, hurst (sin wavelets inicialmente)
  • Epochs reducidos: 15 (en lugar de 30)
  • Lookback reducido: 5 (en lugar de 10)
  • Grid search simplificado: 2 configuraciones ANN
  • Checkpoints cada 5 combinaciones
  • Sampling de datos cada 2 puntos para ML
🚀 INICIANDO BENCHMARK OPTIMIZADO (8-12 HORAS)
   ℹ️ No se encontraron checkpoints previos
📊 CONFIGURACIÓN OPTIMIZADA:
   Features: ['DIX', 'GEX', 'SKEW']
   Modelos ML: ['ANN', 'LSTM', 'GRU']
   Modelos Estadísticos: ['ARIMA']
   Métodos Fractales: ['none', 'hurst']
   Combinaciones de Features: 6
   🎯 Total Experimentos: 144
   ⏱️ Tiempo Estimado: 8-12 horas
   🆕 Iniciando análisis completo desde el principio

📊 Cargando datos...
   Datos cargados: 3092 observacio

c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index begin

        ✅ MSE: 18.586946, R²: -8.9923
   Preparando features para régimen con 1285 observaciones
   ✅ Secuencias creadas: 640
      🤖 Entrenando ANN...
      🔍 Grid search rápido para ANN

        ✅ MSE: 0.027261, R²: -12.6732
      🤖 Entrenando LSTM...
        ✅ MSE: 0.024064, R²: -11.0699
      🤖 Entrenando GRU...
        ✅ MSE: 0.029548, R²: -13.8202

   🔬 Features: DIX | Fractal: hurst (2.8%)
      📊 Entrenando ARIMA...


c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index begin

        ✅ MSE: 18.396152, R²: -8.8754
   Preparando features para régimen con 1274 observaciones
   ✅ Secuencias creadas: 635
      🤖 Entrenando ANN...
      🔍 Grid search rápido para ANN
        ✅ MSE: 0.028815, R²: -15.6089
      🤖 Entrenando LSTM...
        ✅ MSE: 0.027548, R²: -14.8783
      🤖 Entrenando GRU...
        ✅ MSE: 0.022479, R²: -11.9568

   🔬 Features: GEX | Fractal: none (5.6%)
      📊 Entrenando ARIMA...
        ✅ MSE: 18.586946, R²: -8.9923
   Preparando features para régimen con 1285 observaciones
   ✅ Secuencias creadas: 640
      🤖 Entrenando ANN...
      🔍 Grid search rápido para ANN


...

        ✅ MSE: 30.100865, R²: -1.2378
   Preparando features para régimen con 1250 observaciones
   ✅ Secuencias creadas: 623
      🤖 Entrenando ANN...
      🔍 Grid search rápido para ANN
        ✅ MSE: 0.033043, R²: -4.9918
      🤖 Entrenando LSTM...
        ✅ MSE: 0.032818, R²: -4.9510
      🤖 Entrenando GRU...
        ✅ MSE: 0.028345, R²: -4.1398

   🔬 Features: GEX+SKEW | Fractal: none (94.4%)
      📊 Entrenando ARIMA...


c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index begin

        ✅ MSE: 19.119252, R²: -0.4216
   Preparando features para régimen con 1261 observaciones
   ✅ Secuencias creadas: 628
      🤖 Entrenando ANN...
      🔍 Grid search rápido para ANN
        ✅ MSE: 0.007300, R²: -0.9510
      🤖 Entrenando LSTM...
        ✅ MSE: 0.007494, R²: -1.0028
      🤖 Entrenando GRU...
        ✅ MSE: 0.012385, R²: -2.3098
   💾 Checkpoint guardado: results/checkpoint_fast_recovery_20250617_003701.pkl

   🔬 Features: GEX+SKEW | Fractal: hurst (97.2%)
      📊 Entrenando ARIMA...


c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\antonio-jose.martine\ai-test\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index begin

        ✅ MSE: 30.100865, R²: -1.2378
   Preparando features para régimen con 1250 observaciones
   ✅ Secuencias creadas: 623
      🤖 Entrenando ANN...
      🔍 Grid search rápido para ANN
        ✅ MSE: 0.019852, R²: -2.5999
      🤖 Entrenando LSTM...
        ✅ MSE: 0.023341, R²: -3.2325
      🤖 Entrenando GRU...
        ✅ MSE: 0.011646, R²: -1.1118
   💾 Checkpoint guardado: results/checkpoint_fast_recovery_COMPLETED_20250617_003726.pkl

💾 GUARDANDO RESULTADOS FINALES...
   ✅ results/optimized_results.csv (144 filas)
   ✅ results/optimized_predictions.csv (18444 filas)
   ✅ results/optimized_regime_characteristics.csv

📊 ANÁLISIS RÁPIDO DE RESULTADOS:
----------------------------------------

🏆 MEJORES MODELOS POR RÉGIMEN:

BULL:
   Mejor: ANN + none
   Features: GEX
   MSE: 0.007815
   R²: -2.9196

BEAR:
   Mejor: ANN + hurst
   Features: GEX+SKEW
   MSE: 0.014316
   R²: 0.3998

RECOVERY:
   Mejor: LSTM + none
   Features: SKEW
   MSE: 0.006713
   R²: -0.7940

🔬 RESUMEN POR MÉTODO FRA